In [6]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 492.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 8.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 19.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 10.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=c336987e6dbf9921447b764e4b3a60ea18640c23ba0a826d9b31afdceca14974
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=83b804e8f643fe73024902470d69dd0aa844ec3bdf631c40e644c62914f4a5b4
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca

In [7]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    with open('/kaggle/input/clickbait-spoiling/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            tweet = i['postText']
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'multi':
                continue
            
            df += [{'text': tweet, 'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)   
            
    
test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')

In [8]:
test_dataset

,text,labels
0,Study shows that most Thais spend majority of ...,False
1,"Xbox One's Terraria Facing This ""Serious Issue""",True
2,The Simple Way To Banish Stinky Garlic Breath,True
3,He Went To Rip Weeds From His Lawn. Hours Late...,False
4,The Dish You Should Never Order in Rome,True
...,...,...
250,"Dog Dies One Hour After Hiking With His Owner,...",False
251,This is what happens when you leave a hotel cl...,False
252,This Texas GOP elector announces that he won't...,True
253,WikiLeaks' Julian Assange Reported Dead,False


In [9]:
train_dataset

,text,labels
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",False
1,NASA sets date for full recovery of ozone hole,True
2,This is what makes employees happy -- and it's...,True
3,The perfect way to cook rice so that it's perf...,True
4,What happens if your new AirPods get lost or s...,False
...,...,...
2636,If You See A Purple Butterfly Sticker At The H...,False
2637,Has Facebook's video explosion completely shak...,False
2638,Cop Is Eating At A Chili's When Teen Hands Him...,False
2639,You need to see this Twitter account that pred...,True


In [10]:
validation_dataset

,text,labels
0,Five Nights at Freddy’s Sequel Delayed for Wei...,False
1,Here’s how much you should be tipping your hai...,True
2,A man swallowed a microSD card and you won't b...,False
3,This popular soda could cure your hangovers sc...,True
4,The anytime snack you won't feel guilty about ...,True
...,...,...
397,Target impresses with new designer collaboration,True
398,Research Reveals Indian Men Are Becoming Infer...,False
399,Jennifer Lawrence Knows Exactly What She Would...,True
400,Hillary Clinton Is Advertising in Deep-Red Neb...,False


# Training

In [44]:
import sklearn

args = {
    "overwrite_output_dir": True,
    "num_train_epochs": 2,
    "fp16": False,
    "gradient_accumulation_steps": 4,
    "evaluate_during_training": True,
    "max_seq_length": 64,
    "learning_rate": 2e-5,
    "early_stopping_consider_epochs": True,
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "acc",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 3,
    "evaluate_during_training_steps": 331,
    "output_dir": "outputs/",
    'warmup_ratio': 0.06,
    # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
    #                    # <> überschreiben sich gegenseitig
    'save_steps': 2000,
    "manual_seed": 12345
}

In [ ]:
# configurations = []

# for learn_rate in [1e-5, 4e-5, 1e-4]:
#     for warumup_ratio in [0.02, 0.06, 0.1]:
#         for batch_size in [8, 16, 32]:
#             configurations += [{
#                 "overwrite_output_dir": True,
#                 "num_train_epochs": 2,
#                 "fp16": False,
#                 "train_batch_size": batch_size,
#                 "gradient_accumulation_steps": 4,
#                 "evaluate_during_training": True,
#                 "max_seq_length": 64,
#                 "learning_rate": learn_rate,
#                 "early_stopping_consider_epochs": True,
#                 "early_stopping_delta": 0.01,
#                 "early_stopping_metric": "acc",
#                 "early_stopping_metric_minimize": False,
#                 "early_stopping_patience": 3,
#                 "evaluate_during_training_steps": 331,
#                 "output_dir": "outputs/",
#                 'warmup_ratio': warumup_ratio,
#                 # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
#                 #                    # <> überschreiben sich gegenseitig
#                 'save_steps': 2000,
#                 "manual_seed": 12345
#             }]

In [39]:
from simpletransformers.classification import ClassificationModel

In [ ]:
# def train_model(config, num):
    
#     config["output_dir"] = "outputs/roberta_" + str(num)
    
#     model = ClassificationModel("roberta", "roberta-large", args = config)
#     # model.train_model(train_dataset)
#     model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

In [ ]:
# for config, num in zip(configurations, range(len(configurations))):
#     print('Running configuration number', num)
    
#     train_model(config, num)

In [40]:
!rm -r outputs/

In [45]:
model = ClassificationModel("roberta", "roberta-large", args = args)
# model.train_model(train_dataset)
model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

  0%|          | 0/2641 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/402 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/402 [00:00<?, ?it/s]

(164,
 defaultdict(list,
             {'global_step': [82, 164],
              'train_loss': [0.7588268518447876, 0.3627997636795044],
              'mcc': [0.377072223973711, 0.5201047498109045],
              'tp': [147, 151],
              'tn': [130, 154],
              'fp': [64, 40],
              'fn': [61, 57],
              'auroc': [0.7017000396510706, 0.8211984536082475],
              'auprc': [0.7035256790671165, 0.8277441421539967],
              'acc': [0.6890547263681592, 0.7587064676616916],
              'eval_loss': [0.6435549855232239, 0.5256767868995667]}))

In [46]:
model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/402 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/51 [00:00<?, ?it/s]

({'mcc': 0.5201047498109045,
  'tp': 151,
  'tn': 154,
  'fp': 40,
  'fn': 57,
  'auroc': 0.8211984536082475,
  'auprc': 0.8277441421539967,
  'acc': 0.7587064676616916,
  'eval_loss': 0.5256767868995667},
 array([[-0.13224807,  0.38399184],
        [-0.25476411,  0.37007114],
        [ 0.24375923, -0.37670895],
        [-0.82091182,  1.27946746],
        [-0.71014428,  0.82397431],
        [ 0.02196617,  0.20596522],
        [ 0.59060568, -0.56214964],
        [ 0.42732379, -0.26778534],
        [-1.33869517,  1.81818044],
        [ 0.58523637, -0.69688654],
        [ 0.11923704, -0.61347204],
        [ 1.2388854 , -1.52684486],
        [ 0.02069198,  0.24040328],
        [ 0.02826235, -0.00766658],
        [ 0.34824958, -0.07681045],
        [-0.22880663, -0.09402707],
        [ 0.77628952, -1.13348186],
        [ 0.71811628, -0.77928776],
        [ 0.63036406, -0.57524616],
        [ 0.02989767, -0.14669064],
        [-0.82618093,  1.10781133],
        [ 0.12327515, -0.20106566],
  

In [47]:
model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/255 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

({'mcc': 0.4011649264361813,
  'tp': 78,
  'tn': 100,
  'fp': 28,
  'fn': 49,
  'auroc': 0.7575049212598426,
  'auprc': 0.7942622246748425,
  'acc': 0.6980392156862745,
  'eval_loss': 0.5759569136425853},
 array([[ 1.48077413e-01, -3.65887165e-01],
        [ 2.43512794e-01, -4.39023852e-01],
        [ 5.21464944e-01, -6.11733556e-01],
        [ 2.45695144e-01, -3.20294052e-01],
        [-4.30467874e-01,  4.56657499e-01],
        [ 2.26931453e-01, -2.66889960e-01],
        [ 3.54547769e-01, -6.30188048e-01],
        [ 2.07698330e-01, -5.15302360e-01],
        [ 3.70623231e-01, -4.22284931e-01],
        [ 4.55101877e-01, -5.50240338e-01],
        [-2.76818201e-02, -2.54544258e-01],
        [-6.87535048e-01,  1.07677853e+00],
        [ 8.20997059e-01, -8.61221969e-01],
        [-1.60337412e+00,  2.16490817e+00],
        [ 2.76180565e-01, -5.47712326e-01],
        [ 5.44833481e-01, -6.55818045e-01],
        [ 2.97371507e-01, -5.07471561e-01],
        [-1.66056991e+00,  2.06835008e+00],
   

# Model Selection with Validation data

In [17]:
from glob import glob
from simpletransformers.classification import ClassificationModel
import sklearn
import pandas as pd
from tqdm import tqdm

df = []

for config, num in zip(configurations, range(len(configurations))):
    directory = "outputs/roberta_" + str(num) + "/"
    
    for checkpoint in tqdm(glob(directory + "checkpoint-*")):
        # print(checkpoint)
        model = ClassificationModel("roberta", checkpoint)
        
        valid_acc = model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        test_acc = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        
        df += [{"checkpoint": checkpoint, "valid_acc": valid_acc, "test_acc": test_acc,
               "config": config}]
        
df = pd.DataFrame(df)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


# Write File

In [20]:
df.to_json('outputs/roberta_eval_results.csv')

In [21]:
print('finished, yay')

finished, yay


In [30]:
df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False)

,checkpoint,valid_acc,test_acc,config
156,outputs/roberta_14/checkpoint-160-epoch-8,0.799087,0.789346,"{'overwrite_output_dir': True, 'num_train_epoc..."
108,outputs/roberta_9/checkpoint-492-epoch-6,0.796043,0.794189,"{'overwrite_output_dir': True, 'num_train_epoc..."
111,outputs/roberta_10/checkpoint-123-epoch-3,0.796043,0.801453,"{'overwrite_output_dir': True, 'num_train_epoc..."
192,outputs/roberta_17/checkpoint-100-epoch-5,0.794521,0.796610,"{'overwrite_output_dir': True, 'num_train_epoc..."
153,outputs/roberta_13/checkpoint-205-epoch-5,0.792998,0.786925,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
245,outputs/roberta_22/checkpoint-41-epoch-1,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
246,outputs/roberta_22/checkpoint-82-epoch-2,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
252,outputs/roberta_22/checkpoint-205-epoch-5,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
254,outputs/roberta_23/checkpoint-140-epoch-7,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [33]:
for _, i in [i for i in df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False).iterrows()][:5]:
    print(i['valid_acc'], i['config'])

0.7990867579908676 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 32, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/roberta_14', 'warmup_ratio': 0.06, 'save_steps': 2000, 'manual_seed': 12345}

0.7960426179604262 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs